In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import embgam
from datasets import load_dataset
import pandas as pd
from toxigen import label_annotations
import numpy as np
import sklearn.metrics
import pickle as pkl

### Load data

In [2]:
TG = load_dataset("skg/toxigen-data", name="annotated", use_auth_token=True)
df_train = label_annotations(pd.DataFrame(TG["train"]))
df_TG_test = pd.DataFrame(TG["test"])
df_test = label_annotations(df_TG_test)

Reusing dataset toxigen-data (/home/chansingh/.cache/huggingface/datasets/skg___toxigen-data/annotated/1.1.0/3dd39bc1508e10d3eebcca2f60948e1529149c78a24594fd929aaa1f1bda74d0)


  0%|          | 0/2 [00:00<?, ?it/s]

### Fit model

In [ ]:
m = embgam.EmbGAMClassifier(
    checkpoint='tomh/toxigen_roberta',
    ngrams=2,
    all_ngrams=False,
)
m.fit(df_train['text'], df_train['label'])
# m.cache_linear_coefs(df_test['text'])
pkl.dump(m, open(f'toxigen_embgam_ngrams=2_roberta.pkl', 'wb'))

### Evaluate performance

In [3]:
m = pkl.load(open('toxigen_embgam_ngrams=2_roberta.pkl', 'rb'))

def get_metrics(m, df):
    preds = m.predict(df['text'], warn=False)
    preds_proba = m.predict_proba(df['text'], warn=False)
    acc = sklearn.metrics.accuracy_score(df['label'], preds)
    roc_auc = sklearn.metrics.roc_auc_score(df['label'], preds_proba[:, 1])
    return acc, roc_auc

acc, roc_auc = get_metrics(m, df_test)
print(f'Test accuracy {acc:0.2f}')
print(f'Test ROC AUC {roc_auc:0.2f}')

Test accuracy 0.68
Test ROC AUC 0.77


In [4]:
rocs = []
target_groups = pd.DataFrame(TG["test"]).target_group.unique()
for target_group in target_groups:
    df = df_test[df_TG_test['target_group'] == target_group]
    try:
        acc, roc_auc = get_metrics(m, df)
        rocs.append(roc_auc)
    except:
        rocs.append(np.nan)
pd.DataFrame.from_dict({'target_group': target_groups, 'roc': rocs})

,target_group,roc
0,black/african-american folks,0.860465
1,black folks / african-americans,0.888889
2,mexican folks,0.702128
3,women,0.849112
4,native american/indigenous folks,0.897436
5,native american folks,NaN
6,folks with physical disabilities,0.678796
7,latino/hispanic folks,0.611529
8,chinese folks,0.777778
9,middle eastern folks,0.836094


## Interpret model

In [6]:
print('Total ngram coefficients: ', len(m.coefs_dict_))
print('Most toxic ngrams')
for k, v in sorted(m.coefs_dict_.items(), key=lambda item: item[1], reverse=True)[:20]:
    print('\t', k, round(v, 2))
print('Least toxic ngrams')
for k, v in sorted(m.coefs_dict_.items(), key=lambda item: item[1])[:8]:
    print('\t', k, round(v, 2))

Total ngram coefficients:  53378
Most toxic ngrams
	 be submissive 1.88
	 sexual degeneracy 1.88
	 never accept 1.67
	 be subservient 1.63
	 evil empire 1.62
	 steal jobs 1.6
	 jewish greed 1.6
	 whites commit 1.6
	 rob you 1.58
	 , toothless 1.56
	 because weak 1.54
	 -destructive 1.52
	 exploiting the 1.52
	 loony bin 1.51
	 , parasitic 1.5
	 , hoodlums 1.49
	 blacks living 1.47
	 typical asiatic 1.44
	 jewish behavior 1.44
	 for steal 1.43
Least toxic ngrams
	 the attitudes -0.9
	 isolation can -0.83
	 human diversity -0.82
	 tribal governments -0.78
	 society\\u2019s perception -0.76
	 participation of\ndisabled -0.76
	 human functioning -0.76
	 at different -0.75


## Worst model errors

In [38]:
df_test['pred'] = m.predict_proba(df_test['text'], warn=False)[:, 1]
df_test['err'] = df_test['label'] - df_test['pred']
# df_test['err'] = df_test['label'] - df_test['pred']
df_test = df_test.sort_values(by='err', ascending=False)
with pd.option_context('display.max_colwidth', 200):
    display(df_test.head(40))

,text,label,err,pred
287,"These guys took offence earlier because I said ""midget"" instead of ""little person"". I was forced to restart my story. I said, ""OK, so I was fucking this dead little person...",1,0.993422,0.006578
501,i wish that i was born in the 1950s so that i could have been a part of the white flight movement and could have moved to the suburbs,1,0.991523,0.008477
164,"There are scholarships available solely for black men, black women, only women, Hispanic men and women...but if there was a scholarship that only a white man could apply for, I bet it would cause ...",1,0.987535,0.012465
882,"Can we talk about how versatile ice is? I mean I can cool down drinks.\nIt can cool down foods and meats.\nIt can round up mexicans.\nI mean just a real versatile item, that ice.",1,0.985652,0.014348
156,"Anerican Dream We were talking about the american dream in class and the teacher said to a german boy if he had a german dream. He said ""We did but no-one liked it.",1,0.981482,0.018518
627,there is a direct correlation between the amount of melanin in the skin of a person and the number of times that person has been in jail,1,0.977739,0.022261
45,"Helen Keller walked into a bar and then a chair, and then a table",1,0.971964,0.028036
381,it is a waste of time and money to try and rehabilitate someone who has been in an institution for a long period of time.,1,0.971438,0.028562
75,"A long time ago, scientists started to combine dog DNA along with human DNA. The outcome turned out to be Mexican girls.",1,0.967118,0.032882
149,Did you hear about the two car pile up in Mexico? 200 Mexicans died.,1,0.965032,0.034968
